In [1]:
import os
import numpy as np
import pickle
import sys
import matplotlib.pyplot as plt
import pandas as pd

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
from astropy.cosmology import Planck15 as cosmo
from astropy.table import Table
from astropy.io import fits

from prospector_utils import *
from prospect import prospect_args
from prospect.fitting import fit_model
from prospect.io import write_results as writer
from prospect.likelihood import NoiseModel
from prospect.likelihood.kernels import Uncorrelated
from prospect.sources.galaxy_basis import CSPSpecBasis
from prospect.utils.obsutils import fix_obs

import prospect.io.read_results as reader
import toolbox_prospector as toolbox

from sedpy.observate import load_filters

import Build


In [223]:
path_wdir = "/Users/amanda/Desktop/Paper/technical/"
path_data = os.path.join(path_wdir, "data/")
path_plots = os.path.join(path_wdir, "plots/")
path_output = os.path.join(path_wdir, "prospector/")
path_flury = os.path.join(path_wdir, 'data/flury/')
path_mock = '/Users/amanda/Desktop/Paper/technical/data/mock/'

In [3]:
objid = 1
obs = Build.build_obs(objid=objid)
model = Build.build_model_w(objid=objid, fit_el=True)
sps= Build.build_sps()
thetas = model.theta.copy()

spec, phot, _ = model.predict(thetas, obs=obs, sps=sps)

phot_unc = np.random.normal(loc=0., scale=np.abs(obs['maggies_unc']))
spec_unc = np.random.normal(loc=0., scale=np.abs(obs['unc']))

phot_final = phot + phot_unc
spec_final = spec + spec_unc

phot_dat = Table([obs['filters'], obs['wave_effective'], phot_final, np.abs(phot_unc), obs['phot_mask']], 
                  names=['filters', 'wave_effective', 'phot', 'phot_unc', 'mask'])
spec_dat = Table([obs['wavelength'], spec_final, np.abs(spec_unc), obs['mask']], 
                  names=['wavelength', 'spectrum', 'spec_unc', 'mask'])

GALAXIE: J003601+003307, ID: 1, PHOTOMETRY: GP_Aperture_Matched_Photometry_v0.fits, EMISSION_LINES: lzlcs_optlines_obs.csv
GALAXIE: J003601+003307, ID: 1, PHOTOMETRY: GP_Aperture_Matched_Photometry_v0.fits, EMISSION_LINES: lzlcs_optlines_obs.csv


In [4]:
phot_file =  path_mock + 'mock_photometry_' + str(objid) + '.dat' #'psd_grid_mocks/' + str(ind+1) + '/mock_photometry_' + str(i) + '.dat'
spec_file =  path_mock + 'mock_spectrum_' + str(objid) + '.dat' #'psd_grid_mocks/' + str(ind+1) + '/mock_spectrum_' + str(i) + '.dat'

phot_dat.write(phot_file, format='ascii', overwrite=True)
spec_dat.write(spec_file, format='ascii', overwrite=True)

for i in range(2,67):
    
    obs = Build.build_obs(objid=i)
    model =  Build.build_model_w(objid=i, fit_el=True)
    sps = Build.build_sps()
    thetas = model.theta.copy()

    spec, phot, _ = model.predict(thetas, obs=obs, sps=sps)

    phot_unc = np.random.normal(loc=0., scale=np.abs(obs['maggies_unc']))
    spec_unc = np.random.normal(loc=0., scale=np.abs(obs['unc']))

    phot_final = phot + phot_unc
    spec_final = spec + spec_unc

    phot_dat = Table([obs['filters'], obs['wave_effective'], phot_final, np.abs(phot_unc), obs['phot_mask']], 
                    names=['filters', 'wave_effective', 'phot', 'phot_unc', 'mask'])
    spec_dat = Table([obs['wavelength'], spec_final, np.abs(spec_unc), obs['mask']], 
                    names=['wavelength', 'spectrum', 'spec_unc', 'mask'])
    
    phot_file =  path_mock + 'mock_photometry_' + str(i) + '.dat'
    spec_file =  path_mock + 'mock_spectrum_' + str(i) + '.dat'
 
    phot_dat.write(phot_file, format='ascii', overwrite=True)
    spec_dat.write(spec_file, format='ascii', overwrite=True)

In [5]:
el_table="lzlcs_optlines_obs.csv"
el = Table.read(os.path.join(path_flury, el_table), format="ascii")
el.add_column([i for i in range(1,67)], name='id', index=0) #uniform reference for name now possible

phot_table='GP_Aperture_Matched_Photometry_v0.fits'
phot_cat = fits.open(os.path.join(path_flury, phot_table))
phot = phot_cat[1].data

thet = Table.read(os.path.join(path_data, 'galaxies_nostring.csv'), format="ascii") 
thet.add_column([i for i in range(1,67)], name='id', index=0)

In [124]:
snr_fuv=[]
snr_nuv=[]
snr_u=[]
snr_g=[]
snr_r=[]
snr_i=[]
snr_z=[]

snr_mag_fuv=[]
snr_mag_nuv=[]
snr_mag_u=[]
snr_mag_g=[]
snr_mag_r=[]
snr_mag_i=[]
snr_mag_z=[]

snr_mag = [snr_mag_fuv,snr_mag_nuv,snr_mag_u,snr_mag_g,snr_mag_r,snr_mag_i,snr_mag_z]
snr = [snr_fuv,snr_nuv,snr_u,snr_g,snr_r,snr_i,snr_z]
fil = ['FUV', 'NUV', 'U', 'G', 'R', 'I', 'Z']

for i in range(0,66):
    for x, s, m in zip(fil, snr, snr_mag):
        if phot['probPSF'][i]==1:
            maggies = 10**((phot['psf_mag_'+x][i]-8.9)/(-2.5))
            maggies_unc = maggies - 10**(((phot['psf_mag_'+x][i]+phot['psf_magerr_'+x][i])-8.9)/(-2.5))
            m.append(maggies/maggies_unc)
            s.append(phot['psf_mag_'+x][i]/phot['psf_magerr_'+x][i])
        else:
            maggies = 10**((phot['aper_mag_3p1_'+x][i]-8.9)/(-2.5))
            maggies_unc = maggies - 10**(((phot['aper_mag_3p1_'+x][i]+phot['aper_magerr_3p1_'+x][i])-8.9)/(-2.5))
            m.append(maggies/maggies_unc)
            s.append(phot['aper_mag_3p1_'+x][i]/phot['aper_magerr_3p1_'+x][i])

snr_tab = Table([snr_mag_fuv,snr_mag_nuv,snr_mag_u,snr_mag_g,snr_mag_r,snr_mag_i,snr_mag_z,snr_fuv,snr_nuv,snr_u,snr_g,snr_r,snr_i,snr_z ], 
                  names=)['snr_mag_fuv','snr_mag_nuv','snr_mag_u','snr_mag_g','snr_mag_r','snr_mag_i','snr_mag_z','snr_fuv','snr_nuv','snr_u','snr_g','snr_r','snr_i','snr_z']

snr_tab.write(path_data +'snr.dat', format='ascii', overwrite=True)         
snr = Table.read(path_data +'snr.dat', format="ascii")


In [ ]:
fig, axs = plt.subplots(2,4, figsize=(35, 18))
mean = []
median= []
for i, j, z in zip(['snr_mag_fuv','snr_mag_nuv','snr_mag_u','snr_mag_g','snr_mag_r','snr_mag_i','snr_mag_z'], [0,1,2,3,0,1,2], [0,0,0,0,1,1,1]):

    dat = snr[i]
    data= np.delete(dat, 8)
    axs[z,j].hist(data, bins=22, color='b')
    axs[z,j].set_xlabel('Signal to Noise Ratio', fontsize=16)
    axs[z,j].set_ylabel('Frequency',fontsize=16)
    axs[z,j].set_title(i,fontsize=16)

    mean_value = np.mean(data)
    median_value = np.median(data)
    mode_value = np.argmax(np.bincount(data.astype(int)))


    mean.append(mean_value)
    median.append(median_value)      

    # Displaying the calculated values
    axs[z,j].axvline(mean_value, color='r', linestyle='--', label='Mean: '+str(np.round(mean_value,2)))
    axs[z,j].axvline(median_value, color='g', linestyle='--', label='Median: '+str(np.round(median_value,2)))
    axs[z,j].axvline(mode_value, color='k', linestyle='--', label='Mode: '+str(np.round(mode_value,2)))
    axs[z,j].legend(fontsize=16)

if axs.size == 8:
    fig.delaxes(axs[1, 3])
plt.tight_layout()

plt.savefig(path_plots+'snr_maggies.pdf')


In [238]:
col_err = []

for i_col in el.columns:
    if ('Ae' in i_col) & ('NAME' not in i_col) & ('id' not in i_col):
        if i_col=='He2r_4686Ae':
            continue
        else:
            col_err.append(i_col)

snr_gal = []
for i in range(0,66):
    x = []
    for j, z in zip(col,col_err):
        if el[j][i]==0:
            continue
        else:
            x.append(el[j][i]/el[z][i])
    snr_gal.append(x)

In [346]:
snr_O2_3726   =  el['O2_3726A']/el['O2_3726Ae']
snr_O2_3729   =  el['O2_3729A']/el['O2_3729Ae']
snr_Ne3_3869  =  el['Ne3_3869A']/el['Ne3_3869Ae']
snr_H1r_3889  =  el['H1r_3889A']/el['H1r_3889Ae']
snr_Ne3_3967  =  el['Ne3_3967A']/el['Ne3_3967Ae']
snr_H1r_3970  =  el['H1r_3970A']/el['H1r_3970Ae']
snr_H1r_4102  =  el['H1r_4102A']/el['H1r_4102Ae']
snr_H1r_4341  =  el['H1r_4341A']/el['H1r_4341Ae']
snr_O3_4363   =  el['O3_4363A']/el['O3_4363Ae']
snr_H1r_4861  =  el['H1r_4861A']/el['H1r_4861Ae']
snr_O3_4959   =  el['O3_4959A']/el['O3_4959Ae']
snr_O3_5007   =  el['O3_5007A']/el['O3_5007Ae']
snr_He1r_5876 =  el['He1r_5876A']/el['He1r_5876Ae']
snr_O1_6300   =  el['O1_6300A']/el['O1_6300Ae']
snr_N2_6548   =  el['N2_6548A']/el['N2_6548Ae']
snr_H1r_6563  =  el['H1r_6563A']/el['H1r_6563Ae']
snr_N2_6584   =  el['N2_6584A']/el['N2_6584Ae']
snr_S2_6716   =  el['S2_6716A']/el['S2_6716Ae']
snr_S2_6731   =  el['S2_6731A']/el['S2_6731Ae']

snr_O2_3726   = np.array(snr_O2_3726  [snr_O2_3726  != -0])
snr_O2_3729   = np.array(snr_O2_3729  [snr_O2_3729  != -0])
snr_Ne3_3869  = np.array(snr_Ne3_3869 [snr_Ne3_3869 != -0])
snr_H1r_3889  = np.array(snr_H1r_3889 [snr_H1r_3889 != -0])
snr_Ne3_3967  = np.array(snr_Ne3_3967 [snr_Ne3_3967 != -0])
snr_H1r_3970  = np.array(snr_H1r_3970 [snr_H1r_3970 != -0])
snr_H1r_4102  = np.array(snr_H1r_4102 [snr_H1r_4102 != -0])
snr_H1r_4341  = np.array(snr_H1r_4341 [snr_H1r_4341 != -0])
snr_O3_4363   = np.array(snr_O3_4363  [snr_O3_4363  != -0])
snr_H1r_4861  = np.array(snr_H1r_4861 [snr_H1r_4861 != -0])
snr_O3_4959   = np.array(snr_O3_4959  [snr_O3_4959  != -0])
snr_O3_5007   = np.array(snr_O3_5007  [snr_O3_5007  != -0])
snr_He1r_5876 = np.array(snr_He1r_5876[snr_He1r_5876!= -0])
snr_O1_6300   = np.array(snr_O1_6300  [snr_O1_6300  != -0])
snr_N2_6548   = np.array(snr_N2_6548  [snr_N2_6548  != -0])
snr_H1r_6563  = np.array(snr_H1r_6563 [snr_H1r_6563 != -0])
snr_N2_6584   = np.array(snr_N2_6584  [snr_N2_6584  != -0])
snr_S2_6716   = np.array(snr_S2_6716  [snr_S2_6716  != -0])
snr_S2_6731   = np.array(snr_S2_6731  [snr_S2_6731  != -0])

In [ ]:
fig, axs = plt.subplots(11,6, figsize=(16, 20))
mean_el_gal = []
median_el_gal = []

for i, j, z in zip(range(0,66), [0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3,4,5], [0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,5,5,5,5,5,5,6,6,6,6,6,6,7,7,7,7,7,7,8,8,8,8,8,8,9,9,9,9,9,9,10,10,10,10,10,10,11,11,11,11,11,11]):

    data = np.array(snr_gal[i])
    fontsize = 6
    axs[z,j].hist(data, bins=10, color='b')
    axs[z,j].set_xlabel('Signal to Noise Ratio', fontsize=fontsize)
    axs[z,j].set_ylabel('Frequency',fontsize=fontsize)
    axs[z,j].set_title(i,fontsize=fontsize)

    mean_value = np.mean(data)
    median_value = np.median(data)
    mode_value = np.argmax(np.bincount(data.astype(int)))

    mean_el_gal.append(mean_value)
    median_el_gal.append(median_value)

    # Displaying the calculated values
    axs[z,j].axvline(mean_value, color='r', linestyle='--', label='Mean: '+str(np.round(mean_value,2)))
    axs[z,j].axvline(median_value, color='g', linestyle='--', label='Median: '+str(np.round(median_value,2)))
    axs[z,j].axvline(mode_value, color='k', linestyle='--', label='Mode: '+str(np.round(mode_value,2)))
    axs[z,j].legend(fontsize=fontsize)

if axs.size == 8:
    fig.delaxes(axs[1, 3])
plt.tight_layout()

plt.savefig(path_plots+'snr_el.pdf')

In [ ]:
el_snr = [snr_O2_3726,snr_O2_3729,snr_Ne3_3869,snr_H1r_3889,snr_Ne3_3967,snr_H1r_3970,snr_H1r_4102,snr_H1r_4341,snr_O3_4363,snr_H1r_4861,snr_O3_4959,snr_O3_5007,snr_He1r_5876,snr_O1_6300,snr_N2_6548,snr_H1r_6563,snr_N2_6584,snr_S2_6716,snr_S2_6731]

In [ ]:
fig, axs = plt.subplots(5, 4, figsize=(12, 10))
mean_el = []
median_el = []

for n, i, j, z in zip(col, el_snr, [0,1,2,3,0,1,2,3,0,1,2,3,0,1,2,3,0,1,2,3], [0,0,0,0,1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4]):

    dat = np.array(i)
    data = [value for value in dat if value != 0]
    data = np.array(data)
    print(data)
    fontsize = 8

    
    axs[z,j].hist(data, bins=40, color='b')
    axs[z,j].set_xlabel('Signal to Noise Ratio', fontsize=fontsize)
    axs[z,j].set_ylabel('Frequency',fontsize=fontsize)
    axs[z,j].set_title(n,fontsize=fontsize)

    mean_value = np.mean(data)
    median_value = np.median(data)
    mode_value = np.argmax(np.bincount(data.astype(int)))

    mean_el.append(mean_value)
    median_el.append(median_value)

    # Displaying the calculated values
    axs[z,j].axvline(mean_value, color='r', linestyle='--', label='Mean: '+str(np.round(mean_value,2)))
    axs[z,j].axvline(median_value, color='g', linestyle='--', label='Median: '+str(np.round(median_value,2)))
    axs[z,j].axvline(mode_value, color='k', linestyle='--', label='Mode: '+str(np.round(mode_value,2)))
    axs[z,j].legend(fontsize=fontsize)

if axs.size == 20:
    fig.delaxes(axs[4, 3])
plt.tight_layout()

plt.savefig(path_plots+'snr_el_line.pdf')

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

fontsize = 12

axs[0,0].plot(range(0,len(mean)     ),  mean     , '.', color = 'b')
axs[0,1].plot(range(0,len(median)   ),  median   , '.', color = 'b')
axs[1,0].plot(range(0,len(mean_el)  ),  mean_el  , '.', color = 'b')
axs[1,1].plot(range(0,len(median_el)),  median_el, '.', color = 'b')

axs[0,0].set_title('mean', fontsize = fontsize)
axs[0,1].set_title('median', fontsize = fontsize)
axs[1,0].set_title('mean_el', fontsize = fontsize)
axs[1,1].set_title('median_el', fontsize = fontsize)

axs[0,0].set_xlabel('Signal to Noise Ratio')
axs[0,1].set_xlabel('Signal to Noise Ratio')
axs[1,0].set_xlabel('Signal to Noise Ratio')
axs[1,1].set_xlabel('Signal to Noise Ratio')

plt.savefig(path_plots+'means_and_median.pdf')

data = {
    'id': ['fav', 'nuv', 'u', 'g', 'r', 'i', 'z','O2_3726A','O2_3729A','Ne3_3869A','H1r_3889A','Ne3_3967A','H1r_3970A','H1r_4102A','H1r_4341A','O3_4363A','H1r_4861A','O3_4959A','O3_5007A','He1r_5876A','O1_6300A','N2_6548A','H1r_6563A','N2_6584A','S2_6716A','S2_6731A']
}
df = pd.DataFrame(data)
snr = df

In [422]:
snr = pd.read_csv(path_data + 'snr.csv')

In [423]:
snr

,id,median,mean
0,fav,5.375269,6.997311
1,nuv,6.284207,8.463580
2,u,15.764481,22.019721
3,g,33.643512,55.249064
4,r,40.311775,61.402708
5,i,20.496393,32.996478
6,z,10.204588,13.868991
7,O2_3726A,26.238455,27.952046
8,O2_3729A,29.616856,30.251134
9,Ne3_3869A,22.342992,25.183171
